<center><b>Описание задачи</b><center>

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).
<br/>
<br/>
<center><b>Входные данные</b><center>

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.
Задание

1.На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

2.Реализуйте два алгоритма рекомендаций:
 - сортировка просмотренных id по популярности (частота появления в просмотренных),
 - сортировка просмотренных id по покупаемости (частота появления в покупках).

3.Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)
<br/>
<br/>  
<center><b>Дополнительные вопросы<b></center>

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.


In [19]:
import re
from collections import Counter, OrderedDict, defaultdict
import pandas as pd

import numpy as np

In [20]:
class OrderedCounter(Counter, OrderedDict):
    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__, OrderedDict(self))

    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)

In [21]:
views_str, purchases_str = re.findall(pattern = "([\d,]+;)([\d,]*\.?)", string="19257,19258,19259,19260,19261,19262,19263,19264,4097,19265;")[0]
# views_str = result[0]
# purchases_str =  None if len(result)<=1 else result[1]
             
             

In [22]:
purchases_str

''

In [23]:
re.findall(pattern="(\d+)", string=views_str)

['19257',
 '19258',
 '19259',
 '19260',
 '19261',
 '19262',
 '19263',
 '19264',
 '4097',
 '19265']

In [24]:
list(map(int, re.findall(pattern="(\d+)", string=views_str)))

[19257, 19258, 19259, 19260, 19261, 19262, 19263, 19264, 4097, 19265]

In [25]:
purchases

[]

In [26]:
# train = []
# with open("./coursera_sessions_train.txt", newline='\n') as f:
#     for line_num, line in enumerate(f):
#         views_str, purchases_str = re.findall(pattern = "([\d,]+;)([\d,]*\.?)", string=line)[0]
#         views = list(map(
#             int, 
#             re.findall(pattern="(\d+)", string=views_str)
#         ))
#         purchases = list(map(
#             int, 
#             re.findall(pattern="(\d+)", string=purchases_str)
#         ))
#         train.append(
#             (
#                 OrderedDict(OrderedCounter(views)), 
#                 OrderedDict(OrderedCounter(purchases))
#             )
#         )

    

In [28]:
# views_frq = defaultdict(int)
# purchases_frq = defaultdict(int)
# with open("./coursera_sessions_train.txt", newline='\n') as f:
#     for line_num, line in enumerate(f):
#         views_str, purchases_str = re.findall(pattern = "([\d,]+;)([\d,]*\.?)", string=line)[0]
#         views = list(map(
#             int, 
#             re.findall(pattern="(\d+)", string=views_str)
#         ))
#         purchases = list(map(
#             int, 
#             re.findall(pattern="(\d+)", string=purchases_str)
#         ))
#         for view in views:
#             views_frq[view] += 1
#         for purchase in purchases:
#             purchases_frq[purchase] += 1

In [126]:
views_frq = Counter()
purchases_frq = Counter()
with open("./coursera_sessions_train.txt", newline='\n') as f:
    for line_num, line in enumerate(f):
        views_str, purchases_str = re.findall(pattern = "([\d,]+;)([\d,]*\.?)", string=line)[0]
        views = list(map(
            int, 
            re.findall(pattern="(\d+)", string=views_str)
        ))
        purchases = list(map(
            int, 
            re.findall(pattern="(\d+)", string=purchases_str)
        ))
        for view in views:
            views_frq.update({view: 1})
        for purchase in purchases:
            purchases_frq.update({purchase: 1})

In [142]:
prec_1 = []
rec_1 = []

prec_5 = []
rec_5 = []

with open("./coursera_sessions_test.txt", newline='\n') as f:
    for line_num, line in enumerate(f):
        views_str, purchases_str = re.findall(pattern = "([\d,]+;)([\d,]*\.?)", string=line)[0]
        views = list(map(
            int, 
            re.findall(pattern="(\d+)", string=views_str)
        ))
        purchases = list(map(
            int, 
            re.findall(pattern="(\d+)", string=purchases_str)
        ))
        v_frq = Counter(views)
        p_frq = Counter(purchases)
        
        pred = list(map(
            lambda x: x[0],
            sorted(
                [
                    (p, purchases_frq[p]) for p in v_frq.keys()
                ],
                reverse=True,
                key=lambda x: x[1]
            )
        ))

        relevant_1 = float(len(set(pred[:1]) & set(p_frq.keys())))
        relevant_5 = float(len(set(pred[:5]) & set(p_frq.keys())))
        n_buy = float(len(p_frq))
        
        if n_buy>0:
            prec_1.append(relevant_1/1.)
            prec_5.append(relevant_5/5.)


            rec_1.append(relevant_1/n_buy)
            rec_5.append(relevant_5/n_buy)
        

In [143]:
np.mean(rec_1)

0.4606201666660298

In [144]:
np.mean(prec_1)

0.5276944065484311

In [145]:
np.mean(rec_5)

0.8201874337490196

In [146]:
np.mean(prec_5)

0.21009549795361532